### **Import & Google Sheet Connection**

In [31]:
import subprocess
import sys
import pandas as pd
import numpy as np
import datetime
import gradio as gr
import folium
import requests
from PIL import Image
import io
import tempfile
import os
import math
import time
from urllib.parse import urlparse
import json
import pytz

In [32]:
def install_packages():
    packages = [
        'gradio',
        'folium',
        'gspread',
        'google-auth',
        'google-auth-oauthlib',
        'google-auth-httplib2'
    ]

    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✅ {package} installed successfully")
        except subprocess.CalledProcessError:
            print(f"❌ Failed to install {package}")

In [33]:
# Google Sheets imports
try:
    import gspread
    from google.oauth2.service_account import Credentials
    from google.auth import default
    from google.colab import auth
    GSPREAD_AVAILABLE = True
    print("✅ Google Sheets libraries imported successfully")
except ImportError as e:
    GSPREAD_AVAILABLE = False
    print(f"⚠️ Google Sheets import failed: {e}")


✅ Google Sheets libraries imported successfully


In [34]:
class ColabGoogleSheetsManager:
    def __init__(self, sheet_id):
        self.sheet_id = sheet_id
        self.csv_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv&gid=0"
        self.last_update = None
        self.cache_duration = 30  # Cache for 30 seconds
        self.data = None

        # Define which columns can be updated and which should be preserved
        self.updatable_columns = [
            'grid_id', 'latitude', 'longitude', 'beacon', 'beacon_count',
            'rescuer', 'day_beacon_dropped', 'reminder_sent', 'reminder_acknowledged'
        ]

        # Columns with formulas that should not be overwritten
        self.formula_columns = [
            'urgency_score', 's_latitude', 's_longtitude', 'timestamp'
        ]

        # Special handling for survivor column
        self.survivor_column = 'survivor'

    @property
    def gc(self):
        return GLOBAL_GC

    @property
    def worksheet(self):
        return GLOBAL_WORKSHEET

    @property
    def is_authenticated(self):
        return AUTH_STATUS == "Connected"

    def get_column_index(self, column_name):
        """Get the column index (1-based) for a given column name"""
        try:
            if not self.worksheet:
                return None

            # Get first row (headers)
            headers = self.worksheet.row_values(1)
            if column_name in headers:
                return headers.index(column_name) + 1  # gspread uses 1-based indexing
            return None
        except Exception as e:
            print(f"Error getting column index for {column_name}: {e}")
            return None

    def update_survivor_column_with_formula_backup(self, df):
      """Fill survivor column with formula instead of literal where applicable."""
      try:
          if not self.is_authenticated or not self.worksheet:
              return False

          survivor_col_idx = self.get_column_index('survivor')
          if not survivor_col_idx:
              print("❌ Survivor column not found.")
              return False

          print("🔄 Updating survivor column with formula preservation...")

          updates = []

          for idx, row in enumerate(df.itertuples(), start=2):
              if getattr(row, 'rescuer') == 1:
                  # If rescuer acknowledged, we overwrite with literal 0
                  updates.append({
                      'range': f'{chr(64 + survivor_col_idx)}{idx}',
                      'values': [[0]]
                  })
              else:
                  # Otherwise, we put back the formula directly
                  formula = f'=IF(AND(Q{idx}="",R{idx}="",),0,1)'  # <- typo here, closing brackets

                  # Correct formula should be:
                  formula = f'=IF(AND(Q{idx}="",R{idx}=""),0,1)'

                  updates.append({
                      'range': f'{chr(64 + survivor_col_idx)}{idx}',
                      'values': [[formula]]
                  })

          if updates:
              self.worksheet.batch_update(updates, value_input_option='USER_ENTERED')
              print(f"✅ Updated {len(updates)} survivor cells.")

          return True

      except Exception as e:
          print(f"❌ Error updating survivor column: {e}")
          return False


    def selective_update_sheet(self, df):
        """Update only specific columns while preserving formulas"""
        try:
            if not self.is_authenticated or not self.worksheet:
                print("❌ Not authenticated - cannot write data to Google Sheets")
                print("📝 Data changes saved locally only")
                self.data = df.copy()
                return False

            print("📝 Performing selective update to Google Sheets...")

            # Get current sheet structure
            headers = self.worksheet.row_values(1)
            if not headers:
                print("❌ Could not read sheet headers")
                return False

            # Prepare batch updates for updatable columns only
            batch_updates = []

            for col_name in self.updatable_columns:
                if col_name not in df.columns:
                    continue

                col_idx = self.get_column_index(col_name)
                if not col_idx:
                    continue

                # Prepare column data (skip header row)
                col_data = []
                for _, row in df.iterrows():
                    if col_name == 'day_beacon_dropped':
                        # Handle date fields
                        value = str(row[col_name]) if row[col_name] is not None else ''
                    else:
                        value = row[col_name]
                    col_data.append([value])

                # Add to batch update (starting from row 2 to preserve headers)
                col_letter = chr(64 + col_idx)  # Convert to A, B, C, etc.
                range_name = f'{col_letter}2:{col_letter}{len(col_data) + 1}'

                batch_updates.append({
                    'range': range_name,
                    'values': col_data
                })

            # Execute batch update for regular columns
            if batch_updates:
                self.worksheet.batch_update(batch_updates, value_input_option='USER_ENTERED')
                print(f"✅ Updated {len(batch_updates)} columns via batch update")

            # Handle survivor column separately with formula preservation
            self.update_survivor_column_with_formula_backup(df)

            # Update local cache
            self.data = df.copy()
            self.last_update = time.time()

            print("✅ Selective update completed successfully")
            return True

        except gspread.exceptions.APIError as e:
            print(f"❌ Google Sheets API error: {e}")
            if "PERMISSION_DENIED" in str(e):
                print("📝 Permission denied - make sure the sheet is shared with your account")
            elif "RATE_LIMIT_EXCEEDED" in str(e):
                print("📝 Rate limit exceeded - please wait before trying again")
            self.data = df.copy()
            return False
        except Exception as e:
            print(f"❌ Error in selective update: {e}")
            print(f"Error details: {str(e)}")
            self.data = df.copy()
            return False

    def read_sheet(self, force_refresh=False):
        """Read data from Google Sheets with caching"""
        try:
            current_time = time.time()
            if (force_refresh or self.last_update is None or
                current_time - self.last_update > self.cache_duration):

                print("🔄 Fetching fresh data from Google Sheets...")

                # Check authentication first
                if not self.is_authenticated:
                    print("⚠️ Not authenticated, using fallback data")
                    return self.create_fallback_data()

                # Try API first, then fall back to CSV
                if self.worksheet:
                    try:
                        records = self.worksheet.get_all_records()
                        if records:
                            df = pd.DataFrame(records)
                            print(f"✅ Loaded {len(df)} records via Google Sheets API")
                        else:
                            print("⚠️ No data found, creating default data")
                            df = self.create_default_sheet_data()
                    except Exception as e:
                        print(f"API read failed, falling back to CSV: {e}")
                        try:
                            df = pd.read_csv(self.csv_url)
                            print(f"✅ Loaded {len(df)} records via CSV export")
                        except Exception as csv_e:
                            print(f"CSV read also failed: {csv_e}")
                            df = self.create_fallback_data()
                else:
                    try:
                        df = pd.read_csv(self.csv_url)
                        print(f"✅ Loaded {len(df)} records via CSV export")
                    except Exception as e:
                        print(f"CSV read failed, creating fallback data: {e}")
                        df = self.create_fallback_data()

                self.last_update = current_time
                self.data = self.process_dataframe(df)
                return self.data.copy()
            else:
                print("📋 Using cached data...")
                return self.data.copy()

        except Exception as e:
            print(f"❌ Error reading from Google Sheets: {e}")
            return self.create_fallback_data()

    def write_to_sheet(self, df):
        """Main write method - now uses selective update"""
        return self.selective_update_sheet(df)

    def full_overwrite_sheet(self, df):
        """Full overwrite method (use only for initial setup)"""
        try:
            if not self.is_authenticated:
                print("❌ Not authenticated - cannot write data to Google Sheets")
                return False

            if not self.worksheet:
                print("❌ Google Sheets API not available - cannot write data")
                return False

            print("📝 Performing FULL overwrite of Google Sheets (USE WITH CAUTION)...")

            # Prepare data for upload
            upload_df = df.copy()

            # Convert date columns to strings for upload
            if 'day_beacon_dropped' in upload_df.columns:
                upload_df['day_beacon_dropped'] = upload_df['day_beacon_dropped'].apply(
                    lambda x: str(x) if x is not None else ''
                )

            # Convert DataFrame to list of lists (including headers)
            data_to_upload = [upload_df.columns.tolist()] + upload_df.values.tolist()

            # Clear existing data and upload new data
            self.worksheet.clear()
            self.worksheet.update(data_to_upload, value_input_option='USER_ENTERED')

            # Update local cache
            self.data = df.copy()
            self.last_update = time.time()

            print("✅ Full overwrite completed")
            return True

        except Exception as e:
            print(f"❌ Error in full overwrite: {e}")
            return False

    def create_default_sheet_data(self):
        """Create default data structure for empty sheets"""
        print("📝 Creating default sheet structure...")
        coordinates, grid_ids = create_1km_grid(base_lat, base_lon, 3)
        n_grids = len(coordinates)

        default_data = {
            'grid_id': grid_ids,
            'latitude': [coord[0] for coord in coordinates],
            'longitude': [coord[1] for coord in coordinates],
            'beacon': [0] * n_grids,
            'beacon_count': [0] * n_grids,
            'rescuer': [0] * n_grids,
            'survivor': [0] * n_grids,
            'day_beacon_dropped': [''] * n_grids,
            'reminder_sent': [0] * n_grids,
            'reminder_acknowledged': [0] * n_grids,
            'urgency_score': [50.0] * n_grids,
            's_latitude': [coord[0] for coord in coordinates],
            's_longtitude': [coord[1] for coord in coordinates],
            'timestamp': [''] * n_grids,
        }

        df = pd.DataFrame(default_data)

        # Initialize the sheet with default data (full overwrite for initial setup)
        if self.is_authenticated and self.worksheet:
            self.full_overwrite_sheet(df)

        return df

    def process_dataframe(self, df):
        """Clean and process the dataframe"""
        # Handle day_beacon_dropped field
        if 'day_beacon_dropped' in df.columns:
            df['day_beacon_dropped'] = df['day_beacon_dropped'].apply(safe_to_date)
        else:
            df['day_beacon_dropped'] = None

        # Ensure all required columns exist
        required_columns = {
            'grid_id': 'G00',
            'latitude': 0.0,
            'longitude': 0.0,
            'beacon': 0,
            'beacon_count': 0,
            'rescuer': 0,
            'survivor': 0,
            'day_beacon_dropped': None,
            'reminder_sent': 0,
            'reminder_acknowledged': 0,
            'urgency_score': 50.0,
            's_latitude': 0.0,
            's_longtitude': 0.0,
            'timestamp': ''
        }

        for col, default_val in required_columns.items():
            if col not in df.columns:
                df[col] = default_val

        # Convert numeric columns
        numeric_columns = ['beacon', 'beacon_count', 'rescuer', 'survivor', 'reminder_sent',
                          'reminder_acknowledged', 'urgency_score', 's_latitude', 's_longtitude']

        for col in numeric_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

        print(f"✅ Processed {len(df)} records from Google Sheets")
        return df

    def create_fallback_data(self):
        """Create fallback data if Google Sheets is unavailable"""
        print("🔧 Creating fallback data...")
        coordinates, grid_ids = create_1km_grid(base_lat, base_lon, 3)
        n_grids = len(coordinates)

        fallback_data = {
            'grid_id': grid_ids,
            'latitude': [coord[0] for coord in coordinates],
            'longitude': [coord[1] for coord in coordinates],
            'beacon': [0] * n_grids,
            'beacon_count': [0] * n_grids,
            'rescuer': np.random.choice([0, 1], size=n_grids, p=[0.8, 0.2]),
            'survivor': [0] * n_grids,
            'day_beacon_dropped': [None] * n_grids,
            'reminder_sent': [0] * n_grids,
            'reminder_acknowledged': [0] * n_grids,
            'urgency_score': np.random.uniform(40, 80, n_grids).round(2),
            's_latitude': [coord[0] for coord in coordinates],
            's_longtitude': [coord[1] for coord in coordinates],
            'timestamp': [''] * n_grids,
        }

        return pd.DataFrame(fallback_data)

    def get_connection_status(self):
        """Get current connection status"""
        if AUTH_STATUS == "Connected":
            return "🟢 Connected (Read/Write)"
        elif AUTH_STATUS == "Authentication Failed":
            return "🔴 Authentication Failed"
        elif AUTH_STATUS == "Sheet Access Failed":
            return "🟡 Sheet Access Failed"
        else:
            return "🟡 Read-Only Mode"

In [35]:
def setup_colab_auth_with_retry(max_retries=3):
    """Setup Google authentication for Colab with retry mechanism"""
    for attempt in range(max_retries):
        try:
            print(f"🔐 Attempting Google authentication (attempt {attempt + 1}/{max_retries})")

            # Authenticate with Google Drive/Sheets
            auth.authenticate_user()
            print("✅ User authentication successful")

            # Get default credentials
            creds, project = default()
            print(f"✅ Default credentials obtained for project: {project}")

            # Create gspread client
            gc = gspread.authorize(creds)
            print("✅ gspread client authorized successfully")

            # Test the connection
            try:
                # Try to access the user's account info to verify connection
                account_info = gc.list_permissions("test")  # This will fail gracefully if no access
            except:
                pass  # Expected to fail, we just want to verify the client works

            print("✅ Google Colab authentication successful")
            return gc, creds

        except Exception as e:
            print(f"❌ Authentication attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                print("⏳ Retrying in 2 seconds...")
                time.sleep(2)
            else:
                print("❌ All authentication attempts failed")
                return None, None

In [36]:
def verify_sheet_access(gc, sheet_id):
    """Verify that we can access the specific Google Sheet"""
    try:
        print(f"🔍 Verifying access to sheet: {sheet_id}")
        sheet = gc.open_by_key(sheet_id)
        worksheet = sheet.sheet1

        # Try to read one cell to verify access
        test_read = worksheet.get('A1')
        print("✅ Sheet access verified successfully")
        return worksheet

    except gspread.exceptions.SpreadsheetNotFound:
        print("❌ Sheet not found. Please check the Sheet ID.")
        print("📝 Make sure the sheet is shared with your Google account or is public")
        return None
    except gspread.exceptions.APIError as e:
        print(f"❌ API Error accessing sheet: {e}")
        print("📝 Make sure you have permission to access this sheet")
        return None
    except Exception as e:
        print(f"❌ Unexpected error accessing sheet: {e}")
        return None


In [37]:
# Google Sheets configuration - UPDATE YOUR SHEET ID HERE
SHEET_ID = "1vv4Sfa65i-Jzxjzp-9d_1EJ9LBnQgmhfC7rr5u8DtOs"  # Replace with your sheet ID
SHEETS_CSV_URL = f"https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid=0"

# Base coordinates - KPPI
base_lat = 3.12074
base_lon = 101.65363

# Global variables for authentication state
GLOBAL_GC = None
GLOBAL_CREDS = None
GLOBAL_WORKSHEET = None
AUTH_STATUS = "Not Authenticated"

In [38]:
def initialize_authentication():
    """Initialize authentication and return status"""
    global GLOBAL_GC, GLOBAL_CREDS, GLOBAL_WORKSHEET, AUTH_STATUS

    if not GSPREAD_AVAILABLE:
        AUTH_STATUS = "Libraries Not Available"
        print("⚠️ Google Sheets libraries not available")
        return False

    print("🚀 Initializing Google Sheets authentication...")

    # Setup authentication
    gc, creds = setup_colab_auth_with_retry()

    if gc is None:
        AUTH_STATUS = "Authentication Failed"
        print("❌ Failed to authenticate with Google")
        return False

    # Verify sheet access
    worksheet = verify_sheet_access(gc, SHEET_ID)

    if worksheet is None:
        AUTH_STATUS = "Sheet Access Failed"
        print("❌ Failed to access Google Sheet")
        return False

    # Store global references
    GLOBAL_GC = gc
    GLOBAL_CREDS = creds
    GLOBAL_WORKSHEET = worksheet
    AUTH_STATUS = "Connected"

    print("🎉 Authentication and sheet access successful!")
    return True

In [39]:
def safe_to_date(val):
    """Safely convert value to date"""
    if val is None or pd.isna(val) or val == 'None':
        return None
    if isinstance(val, datetime.date):
        return val
    try:
        return pd.to_datetime(val).date()
    except:
        return None

In [40]:
# Initialize authentication
print("🚀 Starting authentication process...")
auth_success = initialize_authentication()

# Create sheets manager
sheets_manager = ColabGoogleSheetsManager(SHEET_ID)

# Display initial status
print(f"\n📊 System Status:")
print(f"   Authentication: {AUTH_STATUS}")
print(f"   Connection: {sheets_manager.get_connection_status()}")
print(f"   Sheet ID: {SHEET_ID}")

if not auth_success:
    print("\n⚠️  WARNING: Authentication failed!")
    print("   - The system will work in fallback mode with simulated data")
    print("   - Changes will not be saved to Google Sheets")
    print("   - To fix this, check your Google Sheets permissions and Sheet ID")

🚀 Starting authentication process...
🚀 Initializing Google Sheets authentication...
🔐 Attempting Google authentication (attempt 1/3)
✅ User authentication successful
✅ Default credentials obtained for project: 
✅ gspread client authorized successfully
✅ Google Colab authentication successful
🔍 Verifying access to sheet: 1vv4Sfa65i-Jzxjzp-9d_1EJ9LBnQgmhfC7rr5u8DtOs
✅ Sheet access verified successfully
🎉 Authentication and sheet access successful!

📊 System Status:
   Authentication: Connected
   Connection: 🟢 Connected (Read/Write)
   Sheet ID: 1vv4Sfa65i-Jzxjzp-9d_1EJ9LBnQgmhfC7rr5u8DtOs


In [41]:
def load_fresh_data(force_refresh=False):
    """Load fresh data from Google Sheets"""
    return sheets_manager.read_sheet(force_refresh=force_refresh)

In [42]:
def save_data_to_sheets(df):
    """Save current dataframe to Google Sheets using selective update"""
    try:
        success = sheets_manager.write_to_sheet(df)
        if success:
            print("✅ Data synchronized with Google Sheets")
        else:
            print("⚠️ Data saved locally but not synchronized with Google Sheets")
        return success
    except Exception as e:
        print(f"❌ Error saving to Google Sheets: {e}")
        return False

In [43]:
def force_full_overwrite(df):
    """Force a full overwrite of the sheet (use with caution)"""
    try:
        success = sheets_manager.full_overwrite_sheet(df)
        if success:
            print("✅ Full sheet overwrite completed")
        else:
            print("❌ Full overwrite failed")
        return success
    except Exception as e:
        print(f"❌ Error in full overwrite: {e}")
        return False

In [44]:
def get_system_status():
    """Get comprehensive system status"""
    status = {
        'auth_status': AUTH_STATUS,
        'connection': sheets_manager.get_connection_status(),
        'sheet_id': SHEET_ID,
        'last_update': sheets_manager.last_update,
        'is_authenticated': sheets_manager.is_authenticated
    }
    return status

### **Function**

In [45]:
def timestamped_message(msg, timezone_str="Asia/Kuala_Lumpur"):
    tz = pytz.timezone(timezone_str)
    now = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    return f"[{now}] {msg}"

In [46]:
def create_1km_grid(center_lat, center_lon, grid_size=3):
    """Create a structured grid where each cell is 1km x 1km"""
    km_to_deg_lat = 1.0 / 111.0
    km_to_deg_lon = 1.0 / (111.0 * math.cos(math.radians(center_lat)))

    coords = []
    grid_ids = []

    start_offset = -(grid_size - 1) / 2

    for i in range(grid_size):
        for j in range(grid_size):
            lat_offset = (start_offset + i) * km_to_deg_lat
            lon_offset = (start_offset + j) * km_to_deg_lon

            grid_lat = center_lat + lat_offset
            grid_lon = center_lon + lon_offset

            coords.append((round(grid_lat, 6), round(grid_lon, 6)))
            grid_ids.append(f'G{i+1}{j+1}')

    return coords, grid_ids

In [47]:
def human_detection_alerts():
    """Get active survivor alerts with location info from Google Sheets data"""
    fresh_df = load_fresh_data()
    active_alerts = []

    if 'human_detection' in fresh_df.columns:
        human_detection_df = fresh_df[fresh_df['human_detection'] == 1]
        for _, row in human_detection_df.iterrows():
            alert_info = {
                'gid': row['grid_id'],
                's_latitude': row['s_latitude1'],
                's_longtitude': row['s_longtitude1'],
                'timestamp':row['timestamp1']
            }
            active_alerts.append(alert_info)

    return active_alerts


In [48]:
def get_active_human_alerts():
    """Get active survivor alerts from Google Sheets data"""
    fresh_df = load_fresh_data()
    active_alerts = set()

    # Check for survivor=1 in the data
    if 'human_detection' in fresh_df.columns:
        human_detection_grids = fresh_df[fresh_df['human_detection'] == 1]['grid_id'].tolist()
        active_alerts.update(human_detection_grids)

    return active_alerts

In [49]:
def survivor_alerts():
    """Get active survivor alerts with location info from Google Sheets data"""
    fresh_df = load_fresh_data()
    active_alerts = []

    if 'survivor' in fresh_df.columns:
        survivors_df = fresh_df[fresh_df['survivor'] == 1]
        for _, row in survivors_df.iterrows():
            alert_info = {
                'gid': row['grid_id'],
                's_latitude': row['s_latitude'],
                's_longtitude': row['s_longtitude'],
                'timestamp':row['timestamp']
            }
            active_alerts.append(alert_info)

    return active_alerts


In [50]:
def get_active_survivor_alerts():
    """Get active survivor alerts from Google Sheets data"""
    fresh_df = load_fresh_data()
    active_alerts = set()

    # Check for survivor=1 in the data
    if 'survivor' in fresh_df.columns:
        survivor_grids = fresh_df[fresh_df['survivor'] == 1]['grid_id'].tolist()
        active_alerts.update(survivor_grids)

    return active_alerts

In [51]:
def create_flood_monitoring_map():
    """Create interactive map with real-time data"""
    try:
        # Refresh data from sheets
        fresh_df = load_fresh_data()

        # Get active alerts from data
        active_survivor_alerts = get_active_survivor_alerts()
        active_human_alerts = get_active_human_alerts()

        m = folium.Map(
            location=[base_lat, base_lon],
            zoom_start=12,
            tiles='OpenStreetMap',
            width='100%',
            height=500
        )

        # Draw grid lines
        km_to_deg_lat = 1.0 / 111.0
        km_to_deg_lon = 1.0 / (111.0 * math.cos(math.radians(base_lat)))

        for i in range(4):
            # Vertical lines
            lat_start = base_lat - 1.5 * km_to_deg_lat
            lat_end = base_lat + 1.5 * km_to_deg_lat
            lon_line = base_lon + (i - 1.5) * km_to_deg_lon

            folium.PolyLine(
                locations=[[lat_start, lon_line], [lat_end, lon_line]],
                color='blue',
                weight=2,
                opacity=0.7
            ).add_to(m)

            # Horizontal lines
            lon_start = base_lon - 1.5 * km_to_deg_lon
            lon_end = base_lon + 1.5 * km_to_deg_lon
            lat_line = base_lat + (i - 1.5) * km_to_deg_lat

            folium.PolyLine(
                locations=[[lat_line, lon_start], [lat_line, lon_end]],
                color='blue',
                weight=2,
                opacity=0.7
            ).add_to(m)

        # Add markers with real-time data
        for idx, row in fresh_df.iterrows():
            lat, lon = row['latitude'], row['longitude']
            grid_id = row['grid_id']

            # Determine marker properties based on data from Google Sheets
            # Priority order: survivor alert > human detection > beacon > rescuer > normal
            if grid_id in active_survivor_alerts:
                color = 'red'
                icon = 'exclamation-sign'
                popup_text = f"🚨 SURVIVOR ALERT - Grid {grid_id}<br/>Last Updated: {datetime.datetime.now(pytz.timezone('Asia/Kuala_Lumpur')).strftime('%Y-%m-%d %H:%M:%S')}"
            elif grid_id in active_human_alerts:
                color = 'orange'
                icon = 'eye-open'
                popup_text = f"👤 HUMAN DETECTION ALERT - Grid {grid_id}<br/>Last Updated: {datetime.datetime.now(pytz.timezone('Asia/Kuala_Lumpur')).strftime('%Y-%m-%d %H:%M:%S')}"
            elif row['beacon'] == 1:
                color = 'blue'
                icon = 'info-sign'
                dropped_date = row['day_beacon_dropped'] if row['day_beacon_dropped'] else 'Unknown'
                popup_text = f"📍 Beacon - Grid {grid_id}<br/>Dropped: {dropped_date}<br/>Coverage: 0.5km radius"
            elif row['rescuer'] == 1:
                color = 'green'
                icon = 'user'
                popup_text = f"👨‍🚒 Rescuer - Grid {grid_id}<br/>Status: Active"
            else:
                color = 'gray'
                icon = 'home'
                popup_text = f"Grid {grid_id} (1km²)<br/>Risk Score: {row['urgency_score']}<br/>Water Level: {row['water_level']}m"

            folium.Marker(
                [lat, lon],
                popup=folium.Popup(popup_text, max_width=200),
                tooltip=f'Grid {grid_id} - Click for details',
                icon=folium.Icon(color=color, icon=icon)
            ).add_to(m)

            # Add beacon coverage circle
            if row['beacon'] == 1:
                folium.Circle(
                    location=[lat, lon],
                    radius=500,  # 0.5km radius
                    color='blue',
                    weight=1,
                    fill=True,
                    fillOpacity=0.2,
                    popup=f'Beacon Coverage - Grid {grid_id}<br/>Radius: 0.5km'
                ).add_to(m)

        # Enhanced legend with real-time indicator and system status
        current_time = timestamped_message(" ")
        status = get_system_status()
        legend_html = f'''
        <div style="position: fixed; top: 10px; right: 10px; width: 220px; height: 270px;
                    background-color: white; border:2px solid grey; z-index:9999;
                    font-size:12px; padding: 8px; box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
        <h4>🔄 Live Grid Monitor</h4>
        <p style="font-size:10px; color: green;">Last Update: {timestamped_message(" ")}</p>
        <p style="font-size:10px; color: {'green' if status['is_authenticated'] else 'red'};">
        Status: {status['connection']}</p>
        <p>🔴 Survivor Alert</p>
        <p>🟠 Human Detection Alert</p>
        <p>🔵 Beacon (0.5km radius)</p>
        <p>🟢 Rescuer Present</p>
        <p>⚪ Normal Grid (1km²)</p>
        <p style="font-size:10px; color: blue;">📊 Data: Google Sheets</p>
        </div>
        '''
        m.get_root().html.add_child(folium.Element(legend_html))

        return m._repr_html_()

    except Exception as e:
        status = get_system_status()
        error_html = f'''
        <div style='padding: 20px; text-align: center; border: 2px solid red; margin: 10px;'>
            <h3>🗺️ Map Loading Error</h3>
            <p>Error: {str(e)}</p>
            <p>Status: {status['connection']}</p>
            <p>Authentication: {status['auth_status']}</p>
            <button onclick='location.reload()' style='padding: 10px; background-color: #007bff; color: white; border: none; cursor: pointer;'>
                🔄 Retry Connection
            </button>
        </div>
        '''
        return error_html

def get_map_html():
    """Generate map HTML with real-time data"""
    return create_flood_monitoring_map()

In [52]:
def check_reminders():
    """Check for beacon collection reminders"""
    fresh_df = load_fresh_data()
    tz = pytz.timezone("Asia/Kuala_Lumpur")
    today = datetime.datetime.now(tz).date()
    reminder_list = []

    # Ensure correct types
    fresh_df["reminder_sent"] = fresh_df["reminder_sent"].fillna(0).astype(int)
    fresh_df["reminder_acknowledged"] = fresh_df["reminder_acknowledged"].fillna(0).astype(int)

    for idx, row in fresh_df.iterrows():
        if pd.notnull(row["day_beacon_dropped"]):
            days_passed = (today - row["day_beacon_dropped"]).days
            print(f"[DEBUG] Grid {row['grid_id']} | Days: {days_passed} | reminder_sent: {row['reminder_sent']} | acknowledged: {row['reminder_acknowledged']}")

            if days_passed >= 2 and row["reminder_sent"] == 0:
                fresh_df.at[idx, "reminder_sent"] = 1
                save_data_to_sheets(fresh_df)

            if row["reminder_sent"] == 1 and row["reminder_acknowledged"] == 0:
                msg = f"Grid {row['grid_id']} beacon dropped {days_passed} days ago - Reminder to collect beacon."
                reminder_list.append(timestamped_message(msg, timezone_str="Asia/Kuala_Lumpur"))

    if not reminder_list:
        reminder_list.append(timestamped_message("No reminders at the moment.", timezone_str="Asia/Kuala_Lumpur"))

    return reminder_list

def show_reminder():
    """Display current reminders"""
    return "\n".join(check_reminders())

In [53]:
def show_alerts():
    """Display active survivor alerts and human detection alerts from Google Sheets."""
    active_survivor_alerts = survivor_alerts()
    active_human_detection_alerts = human_detection_alerts()

    # If no alerts at all
    if not active_survivor_alerts and not active_human_detection_alerts:
        return [timestamped_message("✅ No survivor or human detection alerts at the moment.")]

    alerts = []

    # Add survivor alerts
    if active_survivor_alerts:
        survivor_messages = [
            f"🚨 Survivor alert active at Grid {alert['gid']} [{alert['s_latitude']}, {alert['s_longtitude']}] at time {alert['timestamp']}"
            for alert in sorted(active_survivor_alerts, key=lambda x: x['gid'])
        ]
        alerts.extend(survivor_messages)

    # Add human detection alerts
    if active_human_detection_alerts:
        human_detection_messages = [
            f"👤 Human detection alert active at Grid {alert['gid']} [{alert['s_latitude']}, {alert['s_longtitude']}] at time {alert['timestamp']}"
            for alert in sorted(active_human_detection_alerts, key=lambda x: x['gid'])
        ]
        alerts.extend(human_detection_messages)

    return alerts

In [54]:
def show_realtime_data():
    """Display real-time data from Google Sheets"""
    fresh_df = load_fresh_data()
    display_df = fresh_df[[
        "grid_id", "latitude", "longitude", "beacon", "beacon_count", "rescuer",
        "survivor",
        "urgency_score", "reminder_sent", "reminder_acknowledged", "s_latitude", "s_longtitude"
    ]].copy()

    # Add last update timestamp and connection status
    #status = get_system_status()
    #tz = pytz.timezone("Asia/Kuala_Lumpur")
    #now = datetime.datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')

    #display_df.loc[0, 'last_update'] = now

    return display_df

In [55]:
def acknowledge_alert(grid_id):
    """Acknowledge survivor alert or human detection alert"""
    if not grid_id:
        return timestamped_message("Please enter a grid ID.")

    fresh_df = load_fresh_data()
    idx = fresh_df.index[fresh_df["grid_id"] == grid_id].tolist()
    if not idx:
        return timestamped_message(f"Grid {grid_id} not found.")

    i = idx[0]

    # Check for active alerts from Google Sheets
    active_survivor_alerts = get_active_survivor_alerts()
    active_human_alerts = get_active_human_alerts()

    has_survivor_alert = grid_id in active_survivor_alerts
    has_human_alert = grid_id in active_human_alerts

    if has_survivor_alert or has_human_alert:
        # Update the rescuer status in the dataframe
        fresh_df.at[i, "rescuer"] = 1

        alert_types = []

        # Reset survivor status if there's a survivor alert
        if has_survivor_alert:
            fresh_df.at[i, "survivor"] = 0
            alert_types.append("survivor")

        # Reset human detection status if there's a human detection alert
        if has_human_alert:
            fresh_df.at[i, "human_detection"] = 0
            alert_types.append("human detection")

        success = save_data_to_sheets(fresh_df)

        alert_description = " and ".join(alert_types)

        if success:
            return timestamped_message(f"✅ Rescuer acknowledged {alert_description} alert at Grid {grid_id}. Alert removed from map.")
        else:
            return timestamped_message(f"⚠️ Rescuer acknowledged {alert_description} alert at Grid {grid_id}. Warning: Changes may not be saved to Google Sheets.")
    else:
        # Just mark rescuer as present
        fresh_df.at[i, "rescuer"] = 1
        success = save_data_to_sheets(fresh_df)

        if success:
            return timestamped_message(f"✅ Rescuer marked as present at Grid {grid_id}.")
        else:
            return timestamped_message(f"⚠️ Rescuer marked as present at Grid {grid_id}. Warning: Changes may not be saved to Google Sheets.")

In [56]:
def acknowledge_reminder(grid_id):
    """Acknowledge beacon collection"""
    if not grid_id:
        return timestamped_message("Please enter a grid ID.")

    fresh_df = load_fresh_data()
    idx = fresh_df.index[fresh_df["grid_id"] == grid_id].tolist()
    if not idx:
        return timestamped_message(f"Grid {grid_id} not found.")

    i = idx[0]
    if fresh_df.at[i, "reminder_sent"] == 0:
        return timestamped_message(f"No reminder sent for Grid {grid_id} yet.")
    if fresh_df.at[i, "reminder_acknowledged"] == 1:
        return timestamped_message(f"Reminder already acknowledged for Grid {grid_id}.")

    # Update the dataframe
    fresh_df.at[i, "beacon"] = 0
    fresh_df.at[i, "beacon_count"] = 0
    fresh_df.at[i, "reminder_acknowledged"] = 1
    fresh_df.at[i, "day_beacon_dropped"] = None
    fresh_df.at[i, "reminder_sent"] = 0
    save_data_to_sheets(fresh_df)

    return timestamped_message(f"📦 Beacon collected from Grid {grid_id}. Blue dot and coverage radius removed from map. Grid ready for new deployment.")

In [57]:
def collect_early(grid_id):
    """Collect beacon early"""
    if not grid_id:
        return timestamped_message("Please enter a grid ID.")

    fresh_df = load_fresh_data()
    idx = fresh_df.index[fresh_df["grid_id"] == grid_id].tolist()
    if not idx:
        return timestamped_message(f"Grid {grid_id} not found.")

    i = idx[0]
    if fresh_df.at[i, "beacon"] == 0:
        return timestamped_message(f"No active beacon at Grid {grid_id} to collect.")

    # Update the dataframe
    fresh_df.at[i, "beacon"] = 0
    fresh_df.at[i, "beacon_count"] = 0
    fresh_df.at[i, "reminder_acknowledged"] = 1
    fresh_df.at[i, "day_beacon_dropped"] = None
    fresh_df.at[i, "reminder_sent"] = 0
    save_data_to_sheets(fresh_df)

    return timestamped_message(f"⚡ Beacon collected early from Grid {grid_id}. Grid ready for new deployment.")

In [58]:
def manual_beacon_drop(grid_id):
    """Drop beacon manually"""
    if not grid_id:
        return timestamped_message("Please enter a grid ID.")

    fresh_df = load_fresh_data()
    idx = fresh_df.index[fresh_df["grid_id"] == grid_id].tolist()
    if not idx:
        return timestamped_message(f"Grid {grid_id} not found.")

    i = idx[0]

    # Check if survivor or rescuer is present
    if fresh_df.at[i, "survivor"] == 1:
        return timestamped_message(f"Cannot drop beacon at Grid {grid_id}. Human is detected in this grid.")

    if fresh_df.at[i, "human_detection"] == 1:
        return timestamped_message(f"Cannot drop beacon at Grid {grid_id}. Survivor is present in this grid.")

    if fresh_df.at[i, "rescuer"] == 1:
        return timestamped_message(f"Cannot drop beacon at Grid {grid_id}. Rescuer is present in this grid.")

    if fresh_df.at[i, "beacon"] == 1:
        return timestamped_message(f"Beacon already exists at Grid {grid_id}. Only 1 beacon per grid allowed.")

    if fresh_df.at[i, "beacon_count"] >= 1:
        return timestamped_message(f"Grid {grid_id} already has maximum beacons (1/1).")

    # Update the dataframe
    fresh_df.at[i, "beacon"] = 1
    fresh_df.at[i, "beacon_count"] = 1
    fresh_df.at[i, "day_beacon_dropped"] = datetime.date.today()
    fresh_df.at[i, "reminder_sent"] = 0
    fresh_df.at[i, "reminder_acknowledged"] = 0
    save_data_to_sheets(fresh_df)

    return timestamped_message(f"✅ Beacon dropped at Grid {grid_id}. Blue dot with 0.5km coverage added to map. Beacon count: 1/1")

### **Route Planning**

In [59]:
from math import radians, sin, cos, sqrt, atan2
import numpy as np

def plan_route(alpha=1.0, beta=10.0):
    """Plan optimized UAV route using urgency-weighted TSP"""
    fresh_df = load_fresh_data()

    # Filter candidates - exclude grids with beacons, rescuers, human detection, AND survivors
    candidates = fresh_df[
        (fresh_df["beacon"] == 0) &
        (fresh_df["rescuer"] == 0) &
        (fresh_df["human_detection"] == 0) &
        (fresh_df["survivor"] == 0)  # Also exclude survivor grids
    ].copy()

    if len(candidates) <= 0:
        return f"❌ Not enough suitable grids to plan route. Found {len(candidates)} grid(s)."

    # Handle missing or invalid urgency scores
    candidates['urgency_score'] = candidates['urgency_score'].fillna(0)
    candidates = candidates[candidates['urgency_score'] > 0]  # Only include grids with positive urgency

    if len(candidates) <= 0:
        return f"❌ No grids with valid urgency scores found for route planning."

    coords = candidates[['latitude', 'longitude']].values
    urgencies = candidates['urgency_score'].values
    grid_ids = candidates['grid_id'].tolist()

    # Ensure we have valid coordinates
    if np.any(np.isnan(coords)) or np.any(np.isinf(coords)):
        return f"❌ Invalid coordinates found in candidate grids."

    start_index = int(np.argmax(urgencies))  # Start at highest urgency
    route_indices = nearest_neighbor_tsp_with_urgency(coords, urgencies, alpha, beta, start_index)

    if not route_indices:
        return f"❌ Failed to generate route."

    optimized_route = [grid_ids[i] for i in route_indices]

    # Calculate total distance
    total_distance = 0
    for i in range(len(route_indices) - 1):
        total_distance += haversine_distance(coords[route_indices[i]], coords[route_indices[i+1]])

    # Build detailed route info
    detailed_steps = []
    for i, route_idx in enumerate(route_indices):
        step_info = (f"{i+1}. {grid_ids[route_idx]} "
                    f"(urgency={urgencies[route_idx]:.1f}, "
                    f"lat={coords[route_idx][0]:.4f}, "
                    f"lon={coords[route_idx][1]:.4f})")
        detailed_steps.append(step_info)

    return (
          f"🚁 Optimized UAV Route (urgency-weighted TSP):\n"
          f"{' → '.join(optimized_route)}\n\n"
          f"📍 Detailed Steps:\n" + '\n'.join(detailed_steps) +
          f"\n\n📊 Total grids: {len(optimized_route)} | Start at highest urgency: {grid_ids[start_index]}"
    )

def haversine_distance(coord1, coord2):
    """Calculate the Haversine distance between two lat/lon coordinates in kilometers"""
    try:
        R = 6371.0  # Earth radius in km
        lat1, lon1 = map(radians, coord1)
        lat2, lon2 = map(radians, coord2)
        dlat = lat2 - lat1
        dlon = lon2 - lon1

        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        return R * c
    except (ValueError, TypeError):
        return float('inf')  # Return infinity for invalid coordinates

def nearest_neighbor_tsp_with_urgency(coords, urgencies, alpha=3.0, beta=7.0, start_index=0):
    """Urgency-aware TSP using Nearest Neighbor with cost = alpha * distance - beta * urgency"""
    n = len(coords)

    if n <= 1:
        return list(range(n))

    if start_index >= n or start_index < 0:
        start_index = 0

    visited = [False] * n
    route = [start_index]
    visited[start_index] = True
    current_index = start_index

    for _ in range(n - 1):
        best_cost = float('inf')
        next_index = None

        for i in range(n):
            if not visited[i]:
                dist = haversine_distance(coords[current_index], coords[i])

                # Skip if distance calculation failed
                if dist == float('inf'):
                    continue

                cost = alpha * dist - beta * urgencies[i]

                if cost < best_cost:
                    best_cost = cost
                    next_index = i

        # If no valid next node found, break (shouldn't happen with valid data)
        if next_index is None:
            break

        route.append(next_index)
        visited[next_index] = True
        current_index = next_index

    return route

### **Gradio**

In [60]:
# Create Gradio interface
with gr.Blocks(title="Flood Monitoring System - 3x3 Grid", theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🌊 Flood Monitoring System - 1km² Grid Network (3x3)")
    gr.Markdown("Monitor flood conditions with structured 1km×1km grids. Each grid can deploy 1 beacon with 0.5km coverage radius.")
    gr.Markdown("**Survivor alerts are automatically detected from Google Sheets when survivor=1**")

    with gr.Row():
        # Left Column: Manual Operations
        with gr.Column(scale=1):
            gr.Markdown("### 🎛️ Manual Operations")
            gr.Markdown("Available Grids: G11, G12, G13, G21, G22, G23, G31, G32, G33")

            grid_input_manual_drop = gr.Textbox(label="Grid ID for Manual Beacon Drop", placeholder="e.g., G22")
            manual_drop_btn = gr.Button("📍 Drop Beacon Manually", variant="secondary")

        # Center Column: Map
        with gr.Column(scale=2):
            gr.Markdown("### 🗺️ Real-time Monitoring Map - 3×3 Grid System")
            gr.Markdown("🔴 Red = Survivor Alert (from Google Sheets) | 🔵 Blue = Beacon (0.5km) | 🟢 Green = Rescuer | ⚪ Gray = Normal | Each square = 1km²")

            map_display = gr.HTML(label="Flood Monitoring Map", value="<div style='text-align: center; padding: 50px;'>Loading map...</div>")
            refresh_map_btn = gr.Button("🔄 Refresh Map", variant="primary")

        # Right Column: Rescuer Actions
        with gr.Column(scale=1):
            gr.Markdown("### 👨‍🚒 Rescuer Actions")

            grid_input_ack_alert = gr.Textbox(label="Grid ID to Acknowledge Alert", placeholder="e.g., G11")
            ack_alert_btn = gr.Button("✅ Acknowledge Alert", variant="primary")

            grid_input_ack_reminder = gr.Textbox(label="Grid ID for Beacon Collection", placeholder="e.g., G22")
            ack_reminder_btn = gr.Button("📦 Acknowledge Collection", variant="primary")

            grid_input_collect_early = gr.Textbox(label="Grid ID for Early Collection", placeholder="e.g., G33")
            collect_early_btn = gr.Button("⚡ Early Collection", variant="primary")

    with gr.Row():
        # UAV Route and Messages
        with gr.Column():
            gr.Markdown("### 🚁 UAV Control & System Messages")

            route_box = gr.Textbox(label="UAV Flight Route", lines=2)
            plan_route_btn = gr.Button("🗺️ Plan UAV Route")

            output_text = gr.Textbox(label="System Messages", lines=6)
            reminder_btn = gr.Button("🔔 Check Reminders")
            alert_btn = gr.Button("🚨 Check Active Alerts")

    # Bottom section: Real-time Data
    with gr.Column():
        gr.Markdown("### 📊 Real-time Grid Data")
        realtime_btn = gr.Button("📈 Refresh Data")
        output_df = gr.Dataframe(
            label="Grid Status Table",
            wrap=True
        )

    # Event handlers
    def refresh_map():
        return get_map_html()

    def handle_acknowledge_alert(grid_id):
        message = acknowledge_alert(grid_id)
        map_html = get_map_html()
        data = show_realtime_data()
        return message, map_html, "", data

    def handle_beacon_drop(grid_id):
        message = manual_beacon_drop(grid_id)
        map_html = get_map_html()
        data = show_realtime_data()
        return message, map_html, "", data

    def handle_acknowledge_reminder(grid_id):
        message = acknowledge_reminder(grid_id)
        map_html = get_map_html()
        data = show_realtime_data()
        return message, map_html, "", data

    def handle_collect_early(grid_id):
        message = collect_early(grid_id)
        map_html = get_map_html()
        data = show_realtime_data()
        return message, map_html, "", data

    def handle_check_reminders():
        message = "\n".join(check_reminders())
        data = show_realtime_data()
        return message, data

    def handle_check_alerts():
        message = "\n".join(show_alerts())
        data = show_realtime_data()
        return message, data

    def handle_plan_route():
        route = plan_route()
        data = show_realtime_data()
        return route, data

    def handle_refresh_data():
        data = show_realtime_data()
        return data

    # Connect buttons to functions
    refresh_map_btn.click(refresh_map, outputs=map_display)

    manual_drop_btn.click(
        handle_beacon_drop,
        inputs=grid_input_manual_drop,
        outputs=[output_text, map_display, grid_input_manual_drop, output_df]
    )

    ack_alert_btn.click(
        handle_acknowledge_alert,
        inputs=grid_input_ack_alert,
        outputs=[output_text, map_display, grid_input_ack_alert, output_df]
    )

    ack_reminder_btn.click(
        handle_acknowledge_reminder,
        inputs=grid_input_ack_reminder,
        outputs=[output_text, map_display, grid_input_ack_reminder, output_df]
    )

    collect_early_btn.click(
        handle_collect_early,
        inputs=grid_input_collect_early,
        outputs=[output_text, map_display, grid_input_collect_early, output_df]
    )

    reminder_btn.click(
        handle_check_reminders,
        outputs=[output_text, output_df]
    )

    alert_btn.click(
        handle_check_alerts,
        outputs=[output_text, output_df]
    )

    plan_route_btn.click(
        handle_plan_route,
        outputs=[route_box, output_df]
    )

    realtime_btn.click(
        handle_refresh_data,
        outputs=output_df
    )

    # Load initial map
    demo.load(refresh_map, outputs=map_display)

if __name__ == "__main__":
    demo.launch(
        share=True,
        debug=True,
        server_name="0.0.0.0"
    )

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e06193312e254439df.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


🔄 Fetching fresh data from Google Sheets...
✅ Loaded 9 records via Google Sheets API
✅ Processed 9 records from Google Sheets
📋 Using cached data...
📋 Using cached data...
📋 Using cached data...
📋 Using cached data...
📋 Using cached data...
[DEBUG] Grid G21 | Days: 5 | reminder_sent: 0 | acknowledged: 0
📝 Performing selective update to Google Sheets...
✅ Updated 9 columns via batch update
🔄 Updating survivor column with formula preservation...
✅ Updated 9 survivor cells.
✅ Selective update completed successfully
✅ Data synchronized with Google Sheets
📋 Using cached data...
📋 Using cached data...
📋 Using cached data...
📋 Using cached data...
📋 Using cached data...
📋 Using cached data...
[DEBUG] Grid G21 | Days: 5 | reminder_sent: 1 | acknowledged: 0
📋 Using cached data...
📋 Using cached data...
📋 Using cached data...
📋 Using cached data...
🔄 Fetching fresh data from Google Sheets...
✅ Loaded 9 records via Google Sheets API
✅ Processed 9 records from Google Sheets
📋 Using cached data...
